# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/10/8

MMVC_Trainerのリポジトリと事前学習済みモデルを、Google Driveに保存します。

​

In [ ]:
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージとリポジトリを導入します。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
print("log: パッケージのインストールが完了しました。\n")

#MMVC_Trainerリポジトリのクローン
print("log: リポジトリのクローンを開始します。")
##ブランチ一覧取得したいのでno-single-branch, 履歴要らないのでmax depth=1
!git clone --depth=1 --no-single-branch https://github.com/isletennos/MMVC_Trainer/
%cd /content/MMVC_Trainer
print("log: リポジトリのクローンが完了しました。\n")

#branch一覧取得
print("log: ブランチの読み込みを開始します。")
##実行ファイルをリポジトリからダウンロードして実行(仮実装)
##元リポジトリにファイルを含めるなどした方が良いかもしれない
!curl -OL https://raw.githubusercontent.com/Srgr0/MMVC_Trainer_Assets/main/git-branch.sh
!bash /content/MMVC_Trainer/git-branch.sh
##作成されたbranches.txtを開いてPythonのリストを作る
with open("/content/MMVC_Trainer/branches.txt",encoding='utf-8') as inp:
  global branches_list #グローバル指定
  branches_list = inp.readline().split(',')
##Typeの確認(listであればok)
print(type(branches_list))
##内容の確認
print(branches_list)
print("log: ブランチの読み込みが完了しました。\n")

​

In [ ]:
#@title ## 2 セットアップ
#@markdown MMVC_Trainerのセットアップを行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * DirectoryでGoogle Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * BranchとFMD_Methodについては、変更不要です。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * Directoryでローカルのパスを指定してください。
#@markdown * BranchとFMD_Methodについては、変更不要です。

#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

def checkout_repository_branch():
  print("log: リポジトリのチェックアウトを開始します。")
  !git checkout $branch
  print("log: リポジトリのチェックアウトが完了しました。\n")

def move_repository():
  print("log: リポジトリの移動を開始します。")
  !mv "/content/MMVC_Trainer" $directory
  %cd $directory
  print("log: リポジトリの移動が完了しました。\n")

def download_finemodel():
  #gdownを使用
  print("log: fine_modelのダウンロードを開始します。")
  %cd {directory}/MMVC_Trainer/fine_model/
  if finemodel_download_method == "gdown":
    ##gdown Google Driveの共有ファイルのダウンロードに使用
    !pip install gdown
    import gdown
    gdown.download('https://drive.google.com/uc?id=1MRmB-9EL2-xhxIIraVx20qWqyX-8ZMPw', 'G_180000.pth', quiet=False)
    gdown.download('https://drive.google.com/uc?id=1oPbMM7cGz-z0fLgj7DW_wPGRCwvgD2jZ', 'D_180000.pth', quiet=False)
  elif finemodel_download_method == "curl":
    !fileid="1MRmB-9EL2-xhxIIraVx20qWqyX-8ZMPw"; filename="G_180000.pth"; html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}"`; curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${fileid}" -o ${filename}
    !fileid="1oPbMM7cGz-z0fLgj7DW_wPGRCwvgD2jZ"; filename="D_180000.pth"; html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}"`; curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${fileid}" -o ${filename}
  print("log: fine_modelのダウンロードが完了しました。\n")

#---関数終わり---

#---メイン---
def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  global directory_input #グローバル指定
  global branch_input #グローバル指定
  global finemodel_download_method_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  directory_input = widgets.Text(value='/content/drive/MyDrive', placeholder='/content/drive/MyDrive', description='Directory:', disabled=False)
  branch_input = widgets.Dropdown(options=branches_list, value='main', description='Branch:', disabled=False)
  finemodel_download_method_input = widgets.Dropdown(options=["gdown", "curl"], value="gdown", description='FMD_Method:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, directory_input, branch_input, finemodel_download_method_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)


def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global directory #グローバル指定
  directory = directory_input.value
  global platform #グローバル指定
  platform = platform_input.value
  global branch #グローバル指定
  branch = branch_input.value
  global finemodel_download_method #グローバル指定
  finemodel_download_method = finemodel_download_method_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    checkout_repository_branch() #リポジトリのチェックアウト
    move_repository() #リポジトリの移動
    download_finemodel() #fine_modelのダウンロード
  elif platform == "Local":
    checkout_repository_branch() #リポジトリのチェックアウト
    move_repository() #リポジトリの移動
    download_finemodel() #fine_modelのダウンロード

#実行
first_view()


​

In [ ]:
#@title ## 3 確認
#@markdown セルを実行して、以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: MMVC_Trainer
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown
#@markdown Directory: MMVC_Trainer/fine_model
#@markdown D_180000.pth
#@markdown G_180000.pth
#@markdown ```

print("Directory: MMVC_Trainer")
!ls -1 {directory}/MMVC_Trainer
print("\n")
print("Directory: MMVC_Trainer/fine_model")
!ls -1 {directory}/MMVC_Trainer/fine_model